# Customer RFM(Recency Frequency and Monetary value) Segmentation Using K Means Clustering


Data Source: 
  - Catalog: retail_analytics
  - Schema: dlt
  - Table: customer_rfm_gold

In [0]:
%pip install threadpoolctl==3.1.0 --quiet
dbutils.library.restartPython()

##Library Imports

In [0]:

import os
os.environ['THREADPOOLCTL_VERBOSE'] = '0'
os.environ['OMP_NUM_THREADS'] = '1'

import warnings
warnings.filterwarnings('ignore')

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import seaborn as sns

## EDA

In [0]:
df = spark.read.table("retail_analytics.dlt.customer_rfm_gold").toPandas()

### RFM Distribution

In [0]:
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.hist(df['Monetary'], bins=10, color='skyblue', edgecolor='black')
plt.title('Monetary Value Distribution')
plt.xlabel('Monetary Value')
plt.ylabel('Count')

plt.subplot(1, 3, 2)
plt.hist(df['Frequency'], bins=10, color='lightgreen', edgecolor='black')
plt.title('Frequency Distribution')
plt.xlabel('Frequency')
plt.ylabel('Count')

plt.subplot(1, 3, 3)
plt.hist(df['Recency'], bins=20, color='salmon', edgecolor='black')
plt.title('Recency Distribution')
plt.xlabel('Recency')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

### Boxplot

In [0]:
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
sns.boxplot(data=df['Monetary'], color='skyblue')
plt.title('Monetary Value Boxplot')
plt.xlabel('Monetary Value')

plt.subplot(1, 3, 2)
sns.boxplot(data=df['Frequency'], color='lightgreen')
plt.title('Frequency Boxplot')
plt.xlabel('Frequency')

plt.subplot(1, 3, 3)
sns.boxplot(data=df['Recency'], color='salmon')
plt.title('Recency Boxplot')
plt.xlabel('Recency')

plt.tight_layout()
plt.show()

### Separating the Outliers From the Non outliers

K Means clustering does not do well with including outliers

It is sensitive to outliers because:

- It uses squared Euclidean distance in its objective function, which heavily penalizes points far from cluster centers
- Outliers can "pull" cluster centroids away from the true center of the main data groups
- The algorithm tries to minimize within-cluster sum of squares, so extreme values have disproportionate influence

In [0]:
df["Monetary"] = df["Monetary"].astype(float)

M_Q1 = df["Monetary"].quantile(0.25)
M_Q3 = df["Monetary"].quantile(0.75)
M_IQR = M_Q3 - M_Q1

monetary_outliers_df = df[
    (df["Monetary"] > (M_Q3 + 1.5 * M_IQR)) | 
    (df["Monetary"] < (M_Q1 - 1.5 * M_IQR))
].copy()

monetary_outliers_df.describe()

In [0]:
F_Q1 = df['Frequency'].quantile(0.25)
F_Q3 = df['Frequency'].quantile(0.75)
F_IQR = F_Q3 - F_Q1

frequency_outliers_df = df[(df['Frequency'] > (F_Q3 + 1.5 * F_IQR)) | (df['Frequency'] < (F_Q1 - 1.5 * F_IQR))].copy()

frequency_outliers_df.describe()

In [0]:
non_outliers_df = df[(~df.index.isin(monetary_outliers_df.index)) & (~df.index.isin(frequency_outliers_df.index))]

non_outliers_df.describe()

### Boxplot for Non Outliers Dataset

In [0]:
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
sns.boxplot(data=non_outliers_df['Monetary'], color='skyblue')
plt.title('Monetary Value Boxplot')
plt.xlabel('Monetary Value')

plt.subplot(1, 3, 2)
sns.boxplot(data=non_outliers_df['Frequency'], color='lightgreen')
plt.title('Frequency Boxplot')
plt.xlabel('Frequency')

plt.subplot(1, 3, 3)
sns.boxplot(data=non_outliers_df['Recency'], color='salmon')
plt.title('Recency Boxplot')
plt.xlabel('Recency')

plt.tight_layout()
plt.show()

### Plotting Customer RFM

In [0]:
fig = plt.figure(figsize=(8, 8))

ax = fig.add_subplot(projection="3d")

scatter = ax.scatter(non_outliers_df["Monetary"], non_outliers_df["Frequency"], non_outliers_df["Recency"])

ax.set_xlabel('Monetary Value')
ax.set_ylabel('Frequency')
ax.set_zlabel('Recency')

ax.set_title('3D Scatter Plot of Customer Data')

plt.show()

### Applying Standard Scaler

In [0]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
import pandas as pd

scaler = StandardScaler()

scaled_data = scaler.fit_transform(non_outliers_df[["Monetary", "Frequency", "Recency"]])

scaled_data

In [0]:
scaled_data_df = pd.DataFrame(scaled_data, index=non_outliers_df.index, columns=("Monetary", "Frequency", "Recency"))

scaled_data_df

### Scatter Plot After Standard Scaler

In [0]:
fig = plt.figure(figsize=(8, 8))

ax = fig.add_subplot(projection="3d")

scatter = ax.scatter(scaled_data_df["Monetary"], scaled_data_df["Frequency"], scaled_data_df["Recency"])

ax.set_xlabel('Monetary Value')
ax.set_ylabel('Frequency')
ax.set_zlabel('Recency')

ax.set_title('3D Scatter Plot of Customer Data')

plt.show()

### K Means Clustering

Finding the right number of clusters is automated within the script using elbow point and silhoutte scores.

In [0]:
import warnings
import os
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

# Suppress warnings
warnings.filterwarnings('ignore', category=UserWarning, module='threadpoolctl')
os.environ['OMP_NUM_THREADS'] = '1'

def find_elbow_point(k_values, inertia_values):
    """
    Find the elbow point using the "elbow method" - point of maximum curvature
    """
    # Calculate the differences
    diffs = np.diff(inertia_values)
    diffs2 = np.diff(diffs)
    
    # Find the point where the second derivative is maximum (most curvature)
    # Add 2 to account for the fact that we start from k=2 and lose 2 points in double diff
    elbow_idx = np.argmax(diffs2) + 2
    elbow_k = k_values[elbow_idx]
    
    return elbow_k

def calculate_elbow_score(k_values, inertia_values):
    """
    Calculate elbow score using the distance from the line connecting first and last points
    """
    # Normalize the data
    k_norm = np.array(k_values) - k_values[0]
    k_norm = k_norm / k_norm[-1]
    
    inertia_norm = np.array(inertia_values) - inertia_values[-1]
    inertia_norm = inertia_norm / (inertia_values[0] - inertia_values[-1])
    
    # Calculate distance from each point to the line connecting first and last points
    distances = []
    for i in range(len(k_norm)):
        # Distance from point to line (perpendicular distance)
        distance = abs(k_norm[i] + inertia_norm[i] - 1) / np.sqrt(2)
        distances.append(distance)
    
    return distances

def find_optimal_k_combined(scaled_data, max_k=12, silhouette_weight=0.6, elbow_weight=0.4):
    """
    Find optimal k using both silhouette score and elbow method
    """
    k_values = list(range(2, max_k + 1))
    results = []
    
    print("Evaluating different k values...")
    print("-" * 60)
    
    for k in k_values:
        kmeans = KMeans(n_clusters=k, random_state=42, max_iter=1000, n_init='auto')
        cluster_labels = kmeans.fit_predict(scaled_data)
        sil_score = silhouette_score(scaled_data, cluster_labels)
        
        results.append({
            'k': k,
            'silhouette_score': sil_score,
            'inertia': kmeans.inertia_,
            'model': kmeans,
            'labels': cluster_labels
        })
        
        print(f"k={k:2d}: Silhouette={sil_score:.3f}, Inertia={kmeans.inertia_:.0f}")
    
    # Extract values for analysis
    silhouette_scores = [r['silhouette_score'] for r in results]
    inertia_values = [r['inertia'] for r in results]
    
    # Find elbow point using curvature method
    elbow_k_curvature = find_elbow_point(k_values, inertia_values)
    
    # Calculate elbow scores (distance from line)
    elbow_distances = calculate_elbow_score(k_values, inertia_values)
    elbow_k_distance = k_values[np.argmax(elbow_distances)]
    
    # Find best silhouette score
    silhouette_k = k_values[np.argmax(silhouette_scores)]
    
    # Normalize scores for combination
    silhouette_normalized = np.array(silhouette_scores) / max(silhouette_scores)
    elbow_normalized = np.array(elbow_distances) / max(elbow_distances)
    
    # Combined score
    combined_scores = (silhouette_weight * silhouette_normalized + 
                      elbow_weight * elbow_normalized)
    
    optimal_k_combined = k_values[np.argmax(combined_scores)]
    
    print("\n" + "=" * 60)
    print("OPTIMAL K ANALYSIS:")
    print("=" * 60)
    print(f"Best k by Silhouette Score: {silhouette_k} (score: {max(silhouette_scores):.3f})")
    print(f"Best k by Elbow Method (curvature): {elbow_k_curvature}")
    print(f"Best k by Elbow Method (distance): {elbow_k_distance}")
    print(f"Best k by Combined Method: {optimal_k_combined}")
    print(f"  - Silhouette weight: {silhouette_weight}")
    print(f"  - Elbow weight: {elbow_weight}")
    print(f"  - Combined score: {max(combined_scores):.3f}")
    
    return results, {
        'optimal_k': optimal_k_combined,
        'silhouette_k': silhouette_k,
        'elbow_k_curvature': elbow_k_curvature,
        'elbow_k_distance': elbow_k_distance,
        'combined_scores': combined_scores,
        'silhouette_scores': silhouette_scores,
        'inertia_values': inertia_values,
        'elbow_distances': elbow_distances
    }

# Run the combined analysis
all_results, analysis = find_optimal_k_combined(scaled_data_df, max_k=12)

# Get the data for plotting
k_values = [r['k'] for r in all_results]
silhouette_scores = analysis['silhouette_scores']
inertia_values = analysis['inertia_values']
elbow_distances = analysis['elbow_distances']
combined_scores = analysis['combined_scores']

optimal_k = analysis['optimal_k']
silhouette_k = analysis['silhouette_k']
elbow_k_curvature = analysis['elbow_k_curvature']
elbow_k_distance = analysis['elbow_k_distance']

# Create comprehensive plots
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Elbow Plot
axes[0, 0].plot(k_values, inertia_values, marker='o', linewidth=2, markersize=8)
axes[0, 0].scatter(elbow_k_curvature, inertia_values[k_values.index(elbow_k_curvature)], 
                   color='red', s=100, zorder=5, label=f'Elbow (curvature): k={elbow_k_curvature}')
axes[0, 0].scatter(elbow_k_distance, inertia_values[k_values.index(elbow_k_distance)], 
                   color='orange', s=100, zorder=5, label=f'Elbow (distance): k={elbow_k_distance}')
axes[0, 0].set_title('Elbow Method - Inertia vs k')
axes[0, 0].set_xlabel('Number of Clusters (k)')
axes[0, 0].set_ylabel('Inertia')
axes[0, 0].grid(True)
axes[0, 0].legend()
axes[0, 0].set_xticks(k_values)

# 2. Silhouette Plot
axes[0, 1].plot(k_values, silhouette_scores, marker='o', color='orange', linewidth=2, markersize=8)
axes[0, 1].scatter(silhouette_k, max(silhouette_scores), 
                   color='red', s=100, zorder=5, label=f'Best Silhouette: k={silhouette_k}')
axes[0, 1].set_title('Silhouette Score vs k')
axes[0, 1].set_xlabel('Number of Clusters (k)')
axes[0, 1].set_ylabel('Silhouette Score')
axes[0, 1].grid(True)
axes[0, 1].legend()
axes[0, 1].set_xticks(k_values)

# 3. Elbow Distance Plot
axes[1, 0].plot(k_values, elbow_distances, marker='s', color='green', linewidth=2, markersize=8)
axes[1, 0].scatter(elbow_k_distance, max(elbow_distances), 
                   color='red', s=100, zorder=5, label=f'Max Distance: k={elbow_k_distance}')
axes[1, 0].set_title('Elbow Method - Distance from Line')
axes[1, 0].set_xlabel('Number of Clusters (k)')
axes[1, 0].set_ylabel('Distance from Line')
axes[1, 0].grid(True)
axes[1, 0].legend()
axes[1, 0].set_xticks(k_values)

# 4. Combined Score Plot
axes[1, 1].plot(k_values, combined_scores, marker='*', color='purple', linewidth=2, markersize=10)
axes[1, 1].scatter(optimal_k, max(combined_scores), 
                   color='red', s=150, zorder=5, label=f'Optimal k: {optimal_k}')
axes[1, 1].set_title('Combined Score (Silhouette + Elbow)')
axes[1, 1].set_xlabel('Number of Clusters (k)')
axes[1, 1].set_ylabel('Combined Score')
axes[1, 1].grid(True)
axes[1, 1].legend()
axes[1, 1].set_xticks(k_values)

plt.tight_layout()
plt.show()

# Final clustering with optimal k
print(f"\n🔄 Running final clustering with optimal k={optimal_k}...")
final_result = all_results[k_values.index(optimal_k)]
final_cluster_labels = final_result['labels']
final_kmeans = final_result['model']

print(f"\n📊 Final Clustering Results:")
print(f"Optimal k: {optimal_k}")
print(f"Silhouette score: {final_result['silhouette_score']:.3f}")
print(f"Inertia: {final_result['inertia']:.0f}")

# Cluster distribution
unique, counts = np.unique(final_cluster_labels, return_counts=True)
print(f"\nCluster Distribution:")
for cluster_id, count in zip(unique, counts):
    percentage = (count / len(final_cluster_labels)) * 100
    print(f"  Cluster {cluster_id}: {count:4d} samples ({percentage:5.1f}%)")

print(f"\n✅ Combined optimal clustering completed!") 
print(f"Used weights: Silhouette={analysis['silhouette_scores'][k_values.index(optimal_k)]:.3f}, Elbow contribution")

In [0]:
optimal_k = analysis['optimal_k']

# Create and train the final model with optimal k
final_kmeans = KMeans(n_clusters=optimal_k, random_state=42, max_iter=1000, n_init='auto')
final_cluster_labels = final_kmeans.fit_predict(scaled_data_df)

print(f"✅ Final clustering completed with optimal k={optimal_k}")
print(f"Cluster labels shape: {final_cluster_labels.shape}")
print(f"Unique clusters: {np.unique(final_cluster_labels)}")

In [0]:
non_outliers_df["Cluster"] = final_cluster_labels

non_outliers_df

### Scatter plot with the labeled clusters

In [0]:
clusters_data = non_outliers_df[['Monetary', 'Frequency', 'Recency', 'Cluster']].to_dict('records')

In [0]:
import plotly.express as px
fig = px.scatter_3d(non_outliers_df, x='Monetary', y='Frequency', z='Recency', 
                    color='Cluster', title='Interactive 3D Clusters')
fig.show()

In [0]:
import matplotlib.pyplot as plt
import numpy as np

# Get unique cluster labels dynamically
unique_clusters = non_outliers_df['Cluster'].unique()
unique_clusters = unique_clusters[~np.isnan(unique_clusters)]

# Generate colors for all clusters
color_palette = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

cluster_colors = {}
for i, cluster in enumerate(unique_clusters):
    cluster_colors[cluster] = color_palette[i % len(color_palette)]

# Map cluster colors and handle NaN values
colors = non_outliers_df['Cluster'].map(cluster_colors)
colors = colors.fillna('#000000')

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

scatter = ax.scatter(non_outliers_df['Monetary'], 
                     non_outliers_df['Frequency'], 
                     non_outliers_df['Recency'], 
                     c=colors,
                     marker='o')

ax.set_xlabel('Monetary Value')
ax.set_ylabel('Frequency')
ax.set_zlabel('Recency')
ax.set_title(f'3D Scatter Plot - Customer Clusters (k={len(unique_clusters)})')

# Force z-axis label to display
ax.zaxis.set_rotate_label(False)
ax.zaxis.label.set_rotation(90)

# Create legend
legend_elements = []
for cluster in unique_clusters:
    count = np.sum(non_outliers_df['Cluster'] == cluster)
    legend_elements.append(plt.Line2D([0], [0], marker='o', color='w',
                                    markerfacecolor=cluster_colors[cluster],
                                    markersize=10,
                                    label=f'Cluster {int(cluster)} ({count} customers)'))

ax.legend(handles=legend_elements, loc='upper left')

# Ensure proper layout
plt.tight_layout()
plt.show()

In [0]:
overlap_indices = monetary_outliers_df.index.intersection(frequency_outliers_df.index)

monetary_only_outliers = monetary_outliers_df.drop(overlap_indices)
frequency_only_outliers = frequency_outliers_df.drop(overlap_indices)
monetary_and_frequency_outliers = monetary_outliers_df.loc[overlap_indices]

monetary_only_outliers["Cluster"] = -1
frequency_only_outliers["Cluster"] = -2
monetary_and_frequency_outliers["Cluster"] = -3

outlier_clusters_df = pd.concat([monetary_only_outliers, frequency_only_outliers, monetary_and_frequency_outliers])

outlier_clusters_df

In [0]:
full_clustering_df = pd.concat([non_outliers_df, outlier_clusters_df])

full_clustering_df

### Saving To Tables

In [0]:
# Convert Pandas DataFrame to PySpark DataFrame
full_clustering_spark_df = spark.createDataFrame(full_clustering_df)

# Register the PySpark DataFrame as a temporary view
full_clustering_spark_df.createOrReplaceTempView("temp_customer_data")

In [0]:
%sql
-- Cell 1: Create catalog and schema
CREATE CATALOG IF NOT EXISTS retail_analytics;
CREATE SCHEMA IF NOT EXISTS retail.ml;

-- Cell 2: Register DataFrame as temporary view and create main table
-- First register the DataFrame (run in Python cell):
-- non_outliers_df.createOrReplaceTempView("temp_customer_data")

CREATE OR REPLACE TABLE retail_analytics.ml.customer_rfm_kmeans_clustered
USING DELTA
AS SELECT *, current_date() as UpdateDate  FROM temp_customer_data;

-- Cell 3: Create temporary view with update date for history table  
CREATE OR REPLACE TEMPORARY VIEW temp_customer_data_with_date AS
SELECT *, current_date() as UpdateDate 
FROM temp_customer_data;

-- Cell 4: Create history table if it doesn't exist
CREATE TABLE IF NOT EXISTS retail_analytics.ml.customer_rfm_kmeans_clustered_history
USING DELTA
AS SELECT * FROM temp_customer_data_with_date WHERE 1=0; -- Creates empty table with schema

-- Cell 5: Perform merge (upsert) operation
MERGE INTO retail_analytics.ml.customer_rfm_kmeans_clustered_history AS tgt
USING temp_customer_data_with_date AS src
ON tgt.CustomerID = src.CustomerID AND tgt.UpdateDate = src.UpdateDate
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *;